#### Mapped Class Behavior

Across all styles of _mapping_ using the `registry` object, the following behaviors are common.

##### Default Constructor

The `registry` applies a _default constructor_, i.e. `__init__` method, to _all mapped classes_ that __don't explicitly have their own `__init__` method__. The _behavior_ of this method is such that it __provides a convenient keyword constructor__ that _will accept_ as `optional keyword arguments` all the attributes that are named.

In [1]:
from sqlalchemy.orm import registry, Session, declarative_base, relationship
from sqlalchemy import (
    Table, Column, Integer, String, ForeignKey, inspect, create_engine, select
)

In [2]:
Base = declarative_base()

In [3]:
class DeclarativeUser(Base):
    __tablename__ = "declarative_user"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    
    addresses = relationship("Address", back_populates="user")

In [4]:
class Address(Base):
    __tablename__ = "address"
    
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("declarative_user.id"))
    
    user = relationship("DeclarativeUser", back_populates="addresses")

An object of type `DeclarativeUser` above will have a __constructor__ which allows `DeclarativeUser` objects to be created.

In [5]:
u1 = DeclarativeUser(name="some name", fullname="some fullname")

The above _constructor_ may be __customized__ by passing a _Python callable_ to the `registry.constructor` parameter which provides the __desired default `__init__()` behavior__.

The _constructor_ __also applies__ to `imperative mappings`.

In [6]:
mapper_registry = registry()

In [7]:
imperative_user_table = Table(
    "imperative_user",
    mapper_registry.metadata,
    Column("id", Integer, primary_key=True),
    Column("name", String),
)

In [8]:
class ImperativeUser:
    pass

In [9]:
mapper_registry.map_imperatively(ImperativeUser, imperative_user_table)

<Mapper at 0x1b60e1ab520; ImperativeUser>

The above class, _mapped imperatively_ as described at `Imperative Mapping`, will _also feature_ the `default constructor` __associated__ with the `registry`.

##### Runtime Introspection of Mapped classes, Instances and Mappers

A _class_ that is __mapped using `registry`__ will also _feature a few attributes_ that are __common to all mappings__.

* The `__mapper__` attribute will _refer to_ the `Mapper` that is __associated__ with the class.

In [10]:
mapper = DeclarativeUser.__mapper__
print(f"{mapper = }")

mapper = <Mapper at 0x1b60e1b82e0; DeclarativeUser>


This `Mapper` is also what's _returned_ when using the `inspect()` function against the _mapped class_.

In [11]:
mapper = inspect(DeclarativeUser)
print(f"{mapper = }")

mapper = <Mapper at 0x1b60e1b82e0; DeclarativeUser>


* The `__table__` attribute will _refer to_ the `Table`, or _more generically_ to the `FromClause` object, to which the class is __mapped__.

In [12]:
table = DeclarativeUser.__table__
print(f"{table = }")

table = Table('declarative_user', MetaData(), Column('id', Integer(), table=<declarative_user>, primary_key=True, nullable=False), Column('name', String(), table=<declarative_user>), Column('fullname', String(), table=<declarative_user>), schema=None)


This `FromClause` is also what's _returned_ when using the `Mapper.local_table` attribute of the `Mapper`.

In [13]:
table = inspect(DeclarativeUser).local_table
print(f"{table = }")

table = Table('declarative_user', MetaData(), Column('id', Integer(), table=<declarative_user>, primary_key=True, nullable=False), Column('name', String(), table=<declarative_user>), Column('fullname', String(), table=<declarative_user>), schema=None)


For a `single-table inheritance mapping`, where the _class is a subclass_ that __does not have a table of its own__, the `Mapper.local_table` attribute as well as the `.__table__` attribute will be __`None`__. __To retrieve the `"selectable"`__ that is _actually selected_ from during a query for this class, this is __available via__ the `Mapper.selectable` attribute.

In [14]:
table = inspect(DeclarativeUser).selectable
print(f"{table = }")

table = Table('declarative_user', MetaData(), Column('id', Integer(), table=<declarative_user>, primary_key=True, nullable=False), Column('name', String(), table=<declarative_user>), Column('fullname', String(), table=<declarative_user>), schema=None)


##### Inspection of Mapper objects

As illustrated in the previous section, the `Mapper` object is __available from any mapped class__, regardless of method, using the `Runtime Inspection API` system. Using the `inspect()` function, one __can acquire__ the `Mapper` from a _mapped class_.

In [15]:
insp = inspect(DeclarativeUser)
print(f"{insp = }")

insp = <Mapper at 0x1b60e1b82e0; DeclarativeUser>


_Detailed information_ is available including `Mapper.columns`.

In [16]:
print(f"{insp.columns = }")

insp.columns = <sqlalchemy.sql.base.ColumnCollection object at 0x000001B60BA99E80>


This is a _namespace_ that __can be viewed in a list format__ or __via individual names__.

In [17]:
print(f"{list(insp.columns) = }")
print(f"{insp.columns.name = }")

list(insp.columns) = [Column('id', Integer(), table=<declarative_user>, primary_key=True, nullable=False), Column('name', String(), table=<declarative_user>), Column('fullname', String(), table=<declarative_user>)]
insp.columns.name = Column('name', String(), table=<declarative_user>)


_Other namespaces_ include `Mapper.all_orm_descriptors`, which __includes all mapped attributes__ as well as __hybrids__, _association proxies_.

In [18]:
print(f"{insp.all_orm_descriptors = }")
print(f"{insp.all_orm_descriptors.keys() = }")

insp.all_orm_descriptors = <sqlalchemy.util._collections.ImmutableProperties object at 0x000001B60C8C44C0>
insp.all_orm_descriptors.keys() = ['id', 'name', 'fullname', 'addresses']


As well as `Mapper.column_attrs`.

In [19]:
print(f"{list(insp.column_attrs) = }")
print(f"{insp.column_attrs.name = }")
print(f"{insp.column_attrs.name.expression = }")

list(insp.column_attrs) = [<ColumnProperty at 0x1b60e166640; id>, <ColumnProperty at 0x1b60e166940; name>, <ColumnProperty at 0x1b60e166a40; fullname>]
insp.column_attrs.name = <ColumnProperty at 0x1b60e166940; name>
insp.column_attrs.name.expression = Column('name', String(), table=<declarative_user>)


##### Inspection of Mapped Instances

The `inspect()` function also _provides information_ about instances of a `mapped class`. When applied to an instance of a _mapped class_, rather than the class itself, the _object returned_ is known as `InstanceState`, which will _provide links_ to not only the __`Mapper` in use by the class__, but also a __detailed interface__ that provides information on the _state of individual attributes_ within the instance __including__ their `current value` and how this relates to what `their database-loaded value` is.

Given an instance of the `User` class __loaded from the database__.

In [20]:
engine = create_engine("sqlite+pysqlite:///:memory:", echo=True, future=True)
session = Session(engine)

Base.metadata.create_all(engine)

sandy = DeclarativeUser(name="sandy", fullname="Sandy Cheeks")
patrick = DeclarativeUser(name="patrick", fullname="Patrick Star")
squidward = DeclarativeUser(name="squidward", fullname="Squidward Tentacles")
krabs = DeclarativeUser(name="ehkrabs", fullname="Eugene H. Krabs")\

session.add_all([sandy, patrick, squidward, krabs])
session.commit()

2022-10-24 12:28:51,096 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 12:28:51,098 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("declarative_user")
2022-10-24 12:28:51,101 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 12:28:51,106 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("declarative_user")
2022-10-24 12:28:51,113 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 12:28:51,118 INFO sqlalchemy.engine.Engine PRAGMA main.table_info("address")
2022-10-24 12:28:51,120 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 12:28:51,123 INFO sqlalchemy.engine.Engine PRAGMA temp.table_info("address")
2022-10-24 12:28:51,124 INFO sqlalchemy.engine.Engine [raw sql] ()
2022-10-24 12:28:51,127 INFO sqlalchemy.engine.Engine 
CREATE TABLE declarative_user (
	id INTEGER NOT NULL, 
	name VARCHAR, 
	fullname VARCHAR, 
	PRIMARY KEY (id)
)


2022-10-24 12:28:51,128 INFO sqlalchemy.engine.Engine [no key 0.00124s] ()
2022-10-24 12:28:51,130 INFO sqlalchemy.engine.E

In [21]:
u1 = session.scalars(select(DeclarativeUser)).first()
print(f"{u1.name = }")

2022-10-24 12:28:51,228 INFO sqlalchemy.engine.Engine BEGIN (implicit)
2022-10-24 12:28:51,231 INFO sqlalchemy.engine.Engine SELECT declarative_user.id, declarative_user.name, declarative_user.fullname 
FROM declarative_user
2022-10-24 12:28:51,235 INFO sqlalchemy.engine.Engine [generated in 0.00460s] ()
u1.name = 'sandy'


The `inspect()` function will return to us an `InstanceState` object.

In [22]:
insp = inspect(u1)
print(f"{insp = }")

insp = <sqlalchemy.orm.state.InstanceState object at 0x000001B60E231E80>


With this object we can see _elements_ such as the `Mapper`.

In [23]:
print(f"{insp.mapper = }")

insp.mapper = <Mapper at 0x1b60e1b82e0; DeclarativeUser>


and the `Session` to which the object is __attached__, if any.

In [24]:
print(f"{insp.session = }")

insp.session = <sqlalchemy.orm.session.Session object at 0x000001B60E21DE20>


and _information_ about the `current persistence state` for the object.

In [25]:
print(f"{insp.persistent = }")
print(f"{insp.pending = }")

insp.persistent = True
insp.pending = False


`Attribute state information` such as attributes that __have not been loaded__ or __lazy loaded__ (assume _addresses_ refers to a `relationship()` on the _mapped class to a related class_).

In [26]:
print(f"{insp.unloaded = }")

insp.unloaded = {'addresses'}


Information regarding the __current `in-Python` status of attributes__, such as attributes that `have not been modified` _since the last_ `flush`.

In [27]:
print(f"{insp.unmodified = }")

insp.unmodified = {'fullname', 'name', 'id', 'addresses'}


as well as _specific history_ on __modifications to attributes__ since the `last flush`.

In [28]:
print(f"{insp.attrs.fullname.value = }")

insp.attrs.fullname.value = 'Sandy Cheeks'


In [29]:
u1.fullname = "New fullname"
print(f"{insp.attrs.fullname.history = }")

insp.attrs.fullname.history = History(added=['New fullname'], unchanged=(), deleted=['Sandy Cheeks'])
